In [8]:
from cashe import get_train_test_split
from src.model_train import ModelTrainer, encode_label
from src.data_application import train_columns
from src.lstm_encoder import LSTMEmbedding
from src.data_local import MergedDataPreprocessing
import pandas as pd

In [2]:
df_train, df_test = get_train_test_split()

In [3]:
df_train.head(3)

,item_CreatedDate,transaction_RequestId,transaction_PatientAge,transaction_PatientEnGender,item_NameEn,item_Price,item_Status,item_Sequence,item_RequestQuantity,item_ResponseQuantity,transaction_DiagnosisIds,transaction_PhysicianIds,item_ResponseState
0,2023-08-24 15:03:23.404,c4427a02-3291-422d-b2e8-b97bf3d5081d,6,1,Examination,80.00,2,1,1.0,1.0,7,",327",approved
1,2023-08-29 11:48:54.060,e6f4ad37-13b9-4694-a61b-495384fcd245,28,0,Examination,80.00,6,1,1.0,0.0,17,",139",rejected
2,2023-08-29 11:48:54.060,e6f4ad37-13b9-4694-a61b-495384fcd245,28,0,PREGNANCY TEST (BLOOD),175.45,6,2,1.0,0.0,17,",139",rejected


In [4]:
X_train = df_train[train_columns[:-1]];  y_train = df_train[train_columns[-1]]
X_test = df_test[train_columns[:-1]];  y_test = df_test[train_columns[-1]]

In [5]:
X_train.head(3)

,transaction_PatientAge,transaction_PatientEnGender,item_NameEn,item_Price,item_Sequence,item_RequestQuantity,transaction_DiagnosisIds
0,6,1,Examination,80.00,1,1.0,7
1,28,0,Examination,80.00,1,1.0,17
2,28,0,PREGNANCY TEST (BLOOD),175.45,2,1.0,17


In [6]:
y_test = encode_label(y_test)
y_train = encode_label(y_train)
y_test[0]

0

In [10]:
lstm_embedding = LSTMEmbedding()
data_prep = MergedDataPreprocessing(df_train)

In [11]:
X_train = data_prep.column_embedding(X_train,textual_col='item_NameEn')
X_test  = data_prep.column_embedding(X_test,textual_col='item_NameEn')

In [13]:
columns_needed = ['transaction_PatientAge','transaction_PatientEnGender','item_Price','item_Sequence','item_RequestQuantity','transaction_DiagnosisIds','item_NameEn1','item_NameEn2','item_NameEn3','item_NameEn4','item_NameEn5','item_NameEn6','item_NameEn7','item_NameEn8','item_NameEn9', 'item_NameEn10','item_NameEn11', 'item_NameEn12', 'item_NameEn13', 'item_NameEn14','item_NameEn15', 'item_NameEn16']

X_train = X_train[columns_needed]
X_test  = X_test[columns_needed]

In [16]:
X_train

,transaction_PatientAge,transaction_PatientEnGender,item_Price,item_Sequence,item_RequestQuantity,transaction_DiagnosisIds,item_NameEn1,item_NameEn2,item_NameEn3,item_NameEn4,...,item_NameEn7,item_NameEn8,item_NameEn9,item_NameEn10,item_NameEn11,item_NameEn12,item_NameEn13,item_NameEn14,item_NameEn15,item_NameEn16
0,6,1,80.00,1,1.0,7,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,28,0,80.00,1,1.0,17,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,28,0,175.45,2,1.0,17,182,83,8,0,...,0,0,0,0,0,0,0,0,0,0
3,56,0,80.00,1,1.0,14,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,56,0,110.00,2,1.0,14,216,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14993,29,1,17.92,11,1.0,14,448,71,6,196,...,0,0,0,0,0,0,0,0,0,0
14994,0,1,220.00,1,1.0,1,198,329,330,107,...,0,0,0,0,0,0,0,0,0,0
14995,0,1,108.33,2,1.0,1,198,107,57,0,...,0,0,0,0,0,0,0,0,0,0
14996,0,1,211.25,3,1.0,1,254,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
model_runner = ModelTrainer(X_train=X_train[train_columns[:-1]],y_train=y_train,X_test=X_test[train_columns[:-1]],y_test=y_test)

In [15]:
model_runner.train_models()

[LightGBM] [Info] Number of positive: 7480, number of negative: 7518
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001033 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1303
[LightGBM] [Info] Number of data points in the train set: 14998, number of used features: 22
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.498733 -> initscore=-0.005067
[LightGBM] [Info] Start training from score -0.005067


LightGBM, Decision Tree, SGD and Neural Network are trained on dataset.


In [22]:
train_cols = ['transaction_PatientAge','transaction_PatientEnGender','item_Price','item_Sequence','item_RequestQuantity','transaction_DiagnosisIds']

In [26]:
model_runner_old = ModelTrainer(X_train=X_train[train_cols],y_train=y_train,X_test=X_test[train_cols],y_test=y_test)
model_runner_old.train_models()

res_before = model_runner_old.evaluate_models()

[LightGBM] [Info] Number of positive: 7480, number of negative: 7518
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000436 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 359
[LightGBM] [Info] Number of data points in the train set: 14998, number of used features: 6
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.498733 -> initscore=-0.005067
[LightGBM] [Info] Start training from score -0.005067


LightGBM, Decision Tree, SGD and Neural Network are trained on dataset.


In [27]:
res = model_runner.evaluate_models()

In [29]:
print('The result before embedding the Item_Name column are:\n',res_before,'\nThe results after the embedding:\n',res)

The result before embedding the Item_Name column are:
 {'Decision Tree': {'Accuracy': 0.67, 'Precision': 0.66, 'Recall': 0.64}, 'LightGBM': {'Accuracy': 0.7, 'Precision': 0.68, 'Recall': 0.71}, 'SGD Classifier': {'Accuracy': 0.48, 'Precision': 0.48, 'Recall': 1.0}, 'XGBoost': {'Accuracy': 0.7, 'Precision': 0.68, 'Recall': 0.73}, 'Neural Network': {'Accuracy': 0.54, 'Precision': 0.51, 'Recall': 0.87}} 
The results after the embedding:
 {'Decision Tree': {'Accuracy': 0.7, 'Precision': 0.7, 'Recall': 0.68}, 'LightGBM': {'Accuracy': 0.74, 'Precision': 0.71, 'Recall': 0.77}, 'SGD Classifier': {'Accuracy': 0.54, 'Precision': 0.55, 'Recall': 0.22}, 'XGBoost': {'Accuracy': 0.74, 'Precision': 0.71, 'Recall': 0.78}, 'Neural Network': {'Accuracy': 0.66, 'Precision': 0.68, 'Recall': 0.54}}


In [30]:
dt_tran = model_runner.get_decision_tree_feature_importance()
dt_feats = pd.DataFrame(list(dt_tran),columns_needed)

dt_feats[0].sort_values(ascending=False)

transaction_PatientAge         0.272781
item_NameEn1                   0.177105
item_Sequence                  0.152865
item_Price                     0.125745
transaction_DiagnosisIds       0.089265
item_NameEn2                   0.042357
transaction_PatientEnGender    0.041143
item_NameEn3                   0.038265
item_NameEn5                   0.021377
item_NameEn4                   0.013492
item_NameEn7                   0.012617
item_RequestQuantity           0.005821
item_NameEn6                   0.004317
item_NameEn9                   0.001766
item_NameEn12                  0.000336
item_NameEn10                  0.000195
item_NameEn15                  0.000194
item_NameEn13                  0.000189
item_NameEn11                  0.000104
item_NameEn16                  0.000066
item_NameEn8                   0.000000
item_NameEn14                  0.000000
Name: 0, dtype: float64

In [31]:
gbm_tran = model_runner.get_lightgbm_feature_importance()
gbm_feats = pd.DataFrame(list(gbm_tran),columns_needed)

gbm_feats[0].sort_values(ascending=False)

item_Price                     0.236333
transaction_PatientAge         0.170000
item_NameEn1                   0.112333
transaction_DiagnosisIds       0.090000
item_NameEn2                   0.081667
item_Sequence                  0.080667
item_NameEn3                   0.080000
item_NameEn4                   0.052000
item_NameEn5                   0.033667
transaction_PatientEnGender    0.028333
item_RequestQuantity           0.013333
item_NameEn6                   0.012000
item_NameEn7                   0.006333
item_NameEn8                   0.002333
item_NameEn9                   0.000667
item_NameEn10                  0.000333
item_NameEn11                  0.000000
item_NameEn12                  0.000000
item_NameEn13                  0.000000
item_NameEn14                  0.000000
item_NameEn15                  0.000000
item_NameEn16                  0.000000
Name: 0, dtype: float64

In [33]:
nn_tran = model_runner.get_neural_network_feature_importance()
nn_feats = pd.DataFrame(list(nn_tran),columns_needed)

nn_feats[0].sort_values(ascending=False)

item_NameEn9                   2.124079e-51
transaction_DiagnosisIds       1.872086e-56
item_NameEn7                   8.984488e-61
item_RequestQuantity           2.691126e-62
item_Price                     1.365272e-63
item_Sequence                  3.172985e-64
item_NameEn5                   1.215570e-65
item_NameEn10                  2.052051e-67
transaction_PatientEnGender    1.510699e-68
item_NameEn14                  7.747009e-69
item_NameEn15                  9.446943e-70
transaction_PatientAge         4.028168e-73
item_NameEn4                   3.658291e-73
item_NameEn6                   9.132540e-75
item_NameEn1                   1.207858e-76
item_NameEn2                   1.706386e-78
item_NameEn12                  7.180760e-82
item_NameEn13                  3.744880e-84
item_NameEn16                  1.266071e-85
item_NameEn11                  2.220348e-88
item_NameEn8                   8.785854e-90
item_NameEn3                   6.277922e-90
Name: 0, dtype: float64